In [1]:
# install packages
!pip install pytesseract
!pip install pdf2image

You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
# import packages
import pytesseract
from pdf2image import convert_from_path
import os
import re
from datetime import datetime
import pandas as pd

In [2]:
# Function to extract text from a PDF file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

def extract_text_from_pdf(pdf_path):
    # Convert the PDF to a list of PIL Image objects
    images = convert_from_path(pdf_path)
    text_list = []
    # Loop through each page of the PDF and extract text using Tesseract
    for image in images:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    # Combine text from all pages into a single string
    text = '\n'.join(text_list)
    return text

In [61]:
# Define the regex pattern to match the date
date_pattern = r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
council_pattern = r'(?i)(\b[A-Za-z]+\s(?:Council|Committee|COMMITTEE)\b)'
session_pattern = r'(?P<session>[\w-]+\ssession)'
agenda_pattern = r'(?i)Agenda item (\d+)'
agenda_detail_pattern = r"Agenda item \d+(.*?)(draft resolution|draft(?:\n\n)?resolution)"
footnote_pattern = r'^(\*|\d{1})\s.*'
split_pattern = r'draft resolution|draft(?:\n\n)?resolution'

In [66]:
# Function to classify text into column A to N
def classify_text_to_column(text):
    result = []

    # extract year and date (A and J)
    # Search for the date in the text
    match = re.search(date_pattern, text)
    if match:
        date = match.group(0)
        date_obj = datetime.strptime(date, '%d %B %Y')
        new_date = date_obj.strftime('%m/%d/%Y') #column J
        year = new_date[-4:] #column A
    else:
        date, year = 'N/A'
        print("No date found")
    result.append(year)


    # extract council/committe name (B)
    first_twenty_lines = text.split('\n')[:20]
    
    for line in first_twenty_lines:
        match = re.search(council_pattern, line)
        if match:
            council = match.group(0)
        else:
            council = 'N/A'
            #print("No council found")
    result.append(council)


    # extract session (C)
    match = re.search(session_pattern, text)
    if match:
        session = match.group("session")
    else:
        session = 'N/A'
        print("Session information not found.")
    result.append(session)


    # extract Agenda item (D)
    match = re.search(agenda_pattern, text)
    if match:
        agenda_item = match.group(1)
    else:
        agenda_item = 'N/A'
        print("No match found")
    result.append(agenda_item)


    # extract Agenda detail (E) and countries (F)
    match = re.search(agenda_detail_pattern, text, re.DOTALL)
    if match:
        agenda_item_text = match.group(1).strip()
    parts = agenda_item_text.split("\n\n")
    parts = [s.replace('\n', '') for s in parts]
    result.append(parts[0])
    result.append(parts[1][:-1])


    # delete extracted text
    split_text = re.split(split_pattern, text, flags=re.IGNORECASE)
    remain_text = split_text[1]
    #print(remain_text)


    # extract Body title number (G)
    # Split the text into lines
    lines = remain_text.split('\n')
    # Get the first non-empty line
    first_line = next(line for line in lines if line.strip())
    # Check if the first non-empty character is a number using regular expressions
    match = re.match(r'^\d+', first_line)
    if match:
        # If the first non-empty character is a number, extract it
        number = match.group(0)
    else:
        number = 'N/A'
    result.append(number)


    # Body title (H)
    # split the text into paragraphs
    paragraphs = remain_text.split('\n\n')
    # get the text content of the first paragraph
    first_paragraph = paragraphs[1]
    if first_paragraph != "The General Assembly,":
        first_paragraph = first_paragraph.replace('\n', ' ') 
        result.append(first_paragraph)
    else:
        idx = 1
        result.append('N/A')


    # Body text (I)
    # extract the text content from the second non-empty paragraph
    body_text = ''
    footnote = ''
    for paragraph in paragraphs[idx:]:
        for line in paragraph.split('\n'):
            # exclude lines containing specific patterns
            #print(line)
            match = re.search(footnote, line)
            if match:
                footnote += match.group(0)
            else:
                if not re.search(r'^(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle)', line):
                    body_text += line.strip() + ' '
    result.append(body_text)

    # date (J)
    result.append(date)
    
    # # file name (K) and country (L) will be obtained outside

    # footnote (M)
    result.append(footnote)

    # scanned (N), cannot know 
    if int(year) > 1990:
        scanned = 'yes'
    else:
        scanned = 'no'
    result.append(scanned)
    
    return result

In [54]:
# Path to the folder containing the PDF files
pdf_folder = './Dataset'

# Path to the folder where the CSV files will be saved
csv_folder = './csv_folder'

# csv file column list
column_list = ['year', 'Council', 'Session', 'Agenda item', 'Agenda detail', 'cosponsored countries', 
               'body title number',	'body title detail', 'body text', 'date', 'footnote', 'scanned', 
               'file', 'filecountry']

# Loop through each folder and PDF file and extract text
for country_folder in os.listdir(pdf_folder):
    print('---------------' + country_folder + '---------------' )
    if not os.path.isdir(os.path.join(pdf_folder, country_folder)):
        continue
    csv_path = os.path.join(csv_folder, f"{country_folder}.csv")
    rows = []
    # Loop through each file 
    for pdf_file in os.listdir(os.path.join(pdf_folder, country_folder)):
        if not pdf_file.endswith('.pdf'):
            continue
        # check if this file is old-version
        year = int(pdf_file.split('_')[0])
        if year < 1994:
            continue

        pdf_path = os.path.join(pdf_folder, country_folder, pdf_file).replace('\\', '/')
        print(pdf_path)
        # try:
        text = extract_text_from_pdf(pdf_path)
        text_columns = classify_text_to_column(text)
        row = text_columns + [pdf_file, country_folder] #add column K and L
        rows.append(row)
        print(rows)
        df = pd.DataFrame(rows, columns = column_list)
        df.to_csv(csv_path, index = False) 
        # except:
        #     print(f"Error extracting text from {pdf_path}")
        #     continue

    # Write the rows to the CSV file
    df = pd.DataFrame(rows, columns = column_list)
    df.to_csv(csv_path, index = False)  

---------------Afghanistan---------------
./Dataset/Afghanistan/1994_163336.pdf


UnboundLocalError: local variable 'agenda_item_text' referenced before assignment

In [67]:
classify_text_to_column(text)

['1994',
 'N/A',
 'Forty-ninth session',
 '152',
 'OBSERVER STATUS FOR THE INTERNATIONAL FEDERATION OF RED CROSSAND RED CRESCENT SOCIETIES IN THE GENERAL ASSEMBLY',
 'Afghanistan, Algeria, Antigua _and Barbuda, Armenia,Australia, Austria, Bahamas, Bangladesh, Barbados, Belarus,Belize, Benin, Bhutan, Bolivia, Bulgaria, Burkina Faso,Cambodia, Cameroon, Canada, Cape Verde, Chile, Colombia,Congo, Costa Rica, Céte d’Ivoire, Croatia, Cyprus,Democratic People’s Republic of Korea, Denmark, Djibouti,Dominican Republic, Ecuador, El Salvador, Eritrea, Estonia,Ethiopia, Fiji, Finland, Gambia, Georgia, Ghana, Guatemala,Guinea, Honduras, Hungary, Iceland, Indonesia, Iraq, Jordan,Kazakhstan, Kyrgyzstan, Lao People’s Democratic Republic,Liberia, Libyan Arab Jamahiriya, Liechtenstein, Madagascar,Malawi, Malaysia, Maldives, Mali, Malta, Marshall Islands,Mauritania, Mexico, Micronesia (Federated States of)Mongolia, Morocco, Mozambique, Myanmar, Namibia, Nepal, NewZealand, Norway, Oman, Pakistan, Panama, 

In [60]:
# delete extracted text
split_text = text.split("draft resolution")
remain_text = split_text[0]
print(remain_text)

UNITED
NATIONS A

@) General Assembly oiste,
—— 3

LIMITED

A/49/L.3
17 October 1994

ORIGINAL: ENGLISH

Forty-ninth session
Agenda item 152

OBSERVER STATUS FOR THE INTERNATIONAL FEDERATION OF RED CROSS
AND RED CRESCENT SOCIETIES IN THE GENERAL ASSEMBLY

Afghanistan, Algeria, Antigua _and Barbuda, Armenia,
Australia, Austria, Bahamas, Bangladesh, Barbados, Belarus,
Belize, Benin, Bhutan, Bolivia, Bulgaria, Burkina Faso,
Cambodia, Cameroon, Canada, Cape Verde, Chile, Colombia,
Congo, Costa Rica, Céte d’Ivoire, Croatia, Cyprus,
Democratic People’s Republic of Korea, Denmark, Djibouti,
Dominican Republic, Ecuador, El Salvador, Eritrea, Estonia,
Ethiopia, Fiji, Finland, Gambia, Georgia, Ghana, Guatemala,
Guinea, Honduras, Hungary, Iceland, Indonesia, Iraq, Jordan,
Kazakhstan, Kyrgyzstan, Lao People’s Democratic Republic,
Liberia, Libyan Arab Jamahiriya, Liechtenstein, Madagascar,
Malawi, Malaysia, Maldives, Mali, Malta, Marshall Islands,
Mauritania, Mexico, Micronesia (Federated States of

In [56]:
test_path = './Dataset/Afghanistan/1994_163336.pdf'
extract_text_from_pdf(test_path)

'UNITED\nNATIONS A\n\n@) General Assembly oiste,\n—— 3\n\nLIMITED\n\nA/49/L.3\n17 October 1994\n\nORIGINAL: ENGLISH\n\nForty-ninth session\nAgenda item 152\n\nOBSERVER STATUS FOR THE INTERNATIONAL FEDERATION OF RED CROSS\nAND RED CRESCENT SOCIETIES IN THE GENERAL ASSEMBLY\n\nAfghanistan, Algeria, Antigua _and Barbuda, Armenia,\nAustralia, Austria, Bahamas, Bangladesh, Barbados, Belarus,\nBelize, Benin, Bhutan, Bolivia, Bulgaria, Burkina Faso,\nCambodia, Cameroon, Canada, Cape Verde, Chile, Colombia,\nCongo, Costa Rica, Céte d’Ivoire, Croatia, Cyprus,\nDemocratic People’s Republic of Korea, Denmark, Djibouti,\nDominican Republic, Ecuador, El Salvador, Eritrea, Estonia,\nEthiopia, Fiji, Finland, Gambia, Georgia, Ghana, Guatemala,\nGuinea, Honduras, Hungary, Iceland, Indonesia, Iraq, Jordan,\nKazakhstan, Kyrgyzstan, Lao People’s Democratic Republic,\nLiberia, Libyan Arab Jamahiriya, Liechtenstein, Madagascar,\nMalawi, Malaysia, Maldives, Mali, Malta, Marshall Islands,\nMauritania, Mexico,

In [24]:
text = "Agenda item 94\n\nRIGHT OF PEOPLES TO SELF-DETERMINATION\n\nAfghanistan, Algeria, Bangladesh, Brazil, Cuba, Djibouti,\nEgypt, Gambia, Ghana, Guinea-Bissau, Indonesia, Jordan,\nMalaysia, Mali, Malta, Mauritania, Morocco, Namibia,\nNigeria, Oman, Pakistan, Qatar, Saudi Arabia, Senegal,\nSierra Leone, Tunisia, United Arab Emirates, United Republic\nof Tanzania, Yemen, Zambia and Zimbabwe: revised draft\n\nresolution\n\nThe right of the "
# Clean up text
text = text.strip()
print(text)
# Compile regex pattern
agenda_detail_pattern = re.compile(r"Agenda item \d+(.*?)(:)", re.DOTALL)
#agenda_detail_pattern = r"Agenda item \d+(.*?)(?=draft resolution)"
match = agenda_detail_pattern.search(text)

if match:
    agenda_item_text = match.group(1).strip()
    parts = agenda_item_text.split("\n\n")
    parts = [s.replace('\n', '') for s in parts]
    print(parts)
    print(parts[0])
    print(parts[1][:-1])

Agenda item 94

RIGHT OF PEOPLES TO SELF-DETERMINATION

Afghanistan, Algeria, Bangladesh, Brazil, Cuba, Djibouti,
Egypt, Gambia, Ghana, Guinea-Bissau, Indonesia, Jordan,
Malaysia, Mali, Malta, Mauritania, Morocco, Namibia,
Nigeria, Oman, Pakistan, Qatar, Saudi Arabia, Senegal,
Sierra Leone, Tunisia, United Arab Emirates, United Republic
of Tanzania, Yemen, Zambia and Zimbabwe: revised draft

resolution

The right of the
['RIGHT OF PEOPLES TO SELF-DETERMINATION', 'Afghanistan, Algeria, Bangladesh, Brazil, Cuba, Djibouti,Egypt, Gambia, Ghana, Guinea-Bissau, Indonesia, Jordan,Malaysia, Mali, Malta, Mauritania, Morocco, Namibia,Nigeria, Oman, Pakistan, Qatar, Saudi Arabia, Senegal,Sierra Leone, Tunisia, United Arab Emirates, United Republicof Tanzania, Yemen, Zambia and Zimbabwe']
RIGHT OF PEOPLES TO SELF-DETERMINATION
Afghanistan, Algeria, Bangladesh, Brazil, Cuba, Djibouti,Egypt, Gambia, Ghana, Guinea-Bissau, Indonesia, Jordan,Malaysia, Mali, Malta, Mauritania, Morocco, Namibia,Nigeria, 